In [1]:
#import all necessary modules
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint

# Import API key
import api_keys

# Get FBI API Key
API_KEY = api_keys.fbi_api_key

#Pull data from the website on trends for all crimes

# base url for getting api data
base_url = "https://api.usa.gov/crime/fbi/sapi/"
#Create a list of states in order to request data from each state
stateAbbr = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
#Create a list of variables that we want to explore data for. In this case male and females
variable = 'all'
#declare the start and end year for the time period that we want to look at
since = '2010'
until = '2019'

#create an empty dataframe
crime_data = pd.DataFrame(columns=[])

#Create a for loop to request data for each state and for each variable for the offense that we would like to look at.
for i in stateAbbr:
    desired_endpoint = "api/arrest/states/offense/" + i + '/' + variable + '/' + since + '/' + until
    req = requests.get(base_url + desired_endpoint+'?api_key='+API_KEY).json()
    new = pd.DataFrame(req["data"])
    #Create new columns in the dataframe so that we know what data corresponds to each state and variable.
    new['State'] = i
    #Append the data from each request to the dataframe
    crime_data = crime_data.append(new)   

In [2]:
#look at the data
crime_data.head()

,value,data_year,month_num,key,State
0,2678,2010,0,Aggravated Assault,AL
1,4,2011,0,Aggravated Assault,AL
2,7,2012,0,Aggravated Assault,AL
3,10,2013,0,Aggravated Assault,AL
4,40,2014,0,Aggravated Assault,AL


In [3]:
#Pull data from the website on employment of officers by state
#https://api.usa.gov/crime/fbi/sapi/api/police-employment/states/MO/2010/2019?API_KEY=iiHnOKfno2Mgkt5AynpvPpUQTEyxE77jo1RU8PIv


# base url for getting api data
base_url = "https://api.usa.gov/crime/fbi/sapi/"
#Create a list of states in order to request data from each state
stateAbbr = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']
#declare the start and end year for the time period that we want to look at
since = '2010'
until = '2019'

#create an empty dataframe
employment_data = pd.DataFrame(columns=[])

#Create a for loop to request data for each state and for each variable for the offense that we would like to look at.
for i in stateAbbr:
    desired_endpoint = "api/police-employment/states/" + i + '/' + since + '/' + until
    req = requests.get(base_url + desired_endpoint+'?api_key='+API_KEY).json()
    new = pd.DataFrame(req["results"])
    #Create new columns in the dataframe so that we know what data corresponds to each state and variable.
    #new['State'] = i
    #Append the data from each request to the dataframe
    employment_data = employment_data.append(new)  

In [4]:
employment_data.head()

,data_year,civilian_ct,female_civilian_ct,female_officer_ct,female_total_ct,male_civilian_ct,male_officer_ct,male_total_ct,population,total_pe_ct,state_id,state_name,state_abbr,agency_count_pe_submitting,csv_header,pe_ct_per_1000
0,2010,5275,3259,796,4055,2016,10134,12150,4642673,16205,2,Alabama,AL,318,None,3.49
1,2011,5410,3395,756,4151,2015,10019,12034,4463637,16185,2,Alabama,AL,325,None,3.63
2,2012,4112,2538,682,3220,1574,7951,9525,3793835,12745,2,Alabama,AL,266,None,3.36
3,2013,4681,2963,719,3682,1718,8773,10491,4255554,14173,2,Alabama,AL,279,None,3.33
4,2014,3757,2359,567,2926,1398,7131,8529,3398260,11455,2,Alabama,AL,228,None,3.37


In [5]:
import psycopg2

# Connecting to the Postgres Server
conn = psycopg2.connect("user=postgres password=122516")
#make it so that every transaction is run independently and a rollback is done automatically if a transaction failed, otherwise a commit occurs.
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT);

In [8]:
# Get cursor object from the database connection
cursor = conn.cursor()

name_Database = "Crime_Data"

# Create the crime data database
sqlCreateDatabase = "create database "+name_Database+";"

# Create a table in PostgreSQL database
cursor.execute(sqlCreateDatabase);

DuplicateDatabase: database "crime_data" already exists


In [11]:
# Connecting to the database
conn = psycopg2.connect("dbname=crime_data user=postgres password=122516")
#make it so that every transaction is run independently and a rollback is done automatically if a transaction failed, otherwise a commit occurs.
conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
cursor = conn.cursor()

In [29]:
#Create a table in the new database

name_Table = "crime_new"

# Create table statement
sqlCreateTable = "create table if not exists "+name_Table+" (value INT NOT null, data_year INT NOT NULL, month_num INT NOT NULL, key VARCHAR ( 100 ) NOT NULL, State CHAR ( 2 ) NOT NUll);"

# Create the table if it doesn't already exists
cursor.execute(sqlCreateTable)

In [30]:
#Function to add the rows of the dataframe into the postgreSQL table
def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

#call and run the function execute many specifying the connection, dataframe and table to add the data to.    
execute_many(conn, crime_data, 'public.crime_new')

execute_many() done
